In [2]:
import pandas as pd
import numpy as np
import collections

path = "/Users/yanbingliu/Dropbox/infocom23-dataset/interm_processed"
interm_path = "/Users/yanbingliu/Dropbox/infocom23-dataset/interm_result"

grid_size = 0.0005
sample_limit = 0
time_limit = 0

unit_lat = 0.0005
unit_lon = 0.0005
digits = 4

operator_list = ["A", "V", "T"]
#operator_list = ["A", "T"]
#operator_list = ["V"]
region_list=["A-R1", "A-R2", "A-R3", "A-R4", "A-R5", "V-R6", "V-R7", "V-R8", "V-R9", "V-R10", "V-R11", "T-R12", "T-R13"]
#region_list=["A-R1", "A-R2", "A-R3", "A-R4", "A-R5"]
#region_list=["A-R1"]
#region_list=["V-R6", "V-R7", "V-R8", "V-R9", "V-R10", "V-R11"]

#ue_type_list = ["Pixel5", "Pixel4a"]
ue_type_list = ["Pixel5"]
#ue_type_list = ["Pixel4a"]
#target_grid_type_list = ["all", "R1A", "R1B", "R1C"]
target_grid_type_list = ["all"]
rat_list = ["5gm", "5gs"]

dataset_stat = []

region_res = {}

rsrp_threshold = -110

thput_sample_thres = 10
dominate_thres = 0.01

thput_factor_list = [0.1,0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9]

omega_list = [0.1, 0.3, 0.5, 0.7, 0.9]
gamma_list = [10, 20, 30, 40, 50]

freq_to_bandwidth = {675:15, 850:20, 1025:5, 5110:10, 5330:10, 9820:10, 9840:5, 
52740:20, 52940:20, 52941:20, 53139:20, 53140:20, 53141:20, 53339:20, 53341:20, 53539:20,
66486:10, 66661:5, 66686:10, 66936:10, 67086:10}

distribution = {}

def get_grid(s, unit):
    return round(int(float(s) / unit) * unit, digits)

for operator in operator_list:
    for ue_type in ue_type_list:
        mccmnc = ""
        if operator.find("A") >= 0:
            mccmnc = "310410"
        elif operator.find("V") >= 0:
            mccmnc = "311480"
        elif operator.find("T") >= 0:
            mccmnc = "310260"

        bw_freq = {}
        lte_bw_freq = {}
            
        total_instance_num = 0
        
        nr_cell_gps = {}
        max_bw_grid = {}
        cell_bw_grid = {}
        cell_set_bw_grid = {}
        
        cell_bw = {}
        
        mw_rsrp_grid = {}
        rsrp_grid = {}
        
        rsrp_list_0_10 = []
        rsrp_list_10_20 = []
        rsrp_list_20_30 = []
        rsrp_list_30_40 = []
        rsrp_list_40_50 = []
        rsrp_list_50_60 = []
        rsrp_list_60_70 = []
        rsrp_list_70_80 = []
        rsrp_list_80_90 = []
        rsrp_list_90_100 = []
        
        rsrp_list_0_20 = []
        rsrp_list_20_40 = []
        rsrp_list_40_60 = []
        rsrp_list_60_80 = []
        rsrp_list_80_100 = []
        
        sub6_rsrp_grid = {}
        
        mw_rsrp_list = []
        sub6_rsrp_list = []
        lte_rsrp_list = []

        for region in region_list:
            if region.find(operator) < 0:
                continue
            
            print(region)
            
            distribution[region] = []
            
            with open(interm_path + "/" + region + "_raw_rss_taskround_list_" + mccmnc + "_all_20210405_20220127.csv.csv", 'r', encoding='utf-8-sig') as lines:
                for line in lines:
                    if line.find("rsrp") >= 0:
                        continue
                    items = line.strip().split(',')
                    #lat = float(items[0])
                    #lon = float(items[1])
                    #gps_str = str(items[0]) + str(items[1])

                    #rsrp_med = float(items[7])
                    #rsrp_0 = float(items[8])
                    #rsrp_10 = float(items[9])
                    #rsrp_25 = float(items[10])
                    #rsrp_50 = float(items[11])
                    #rsrp_75 = float(items[12])
                    #rsrp_90 = float(items[13])
                    #rsrp_100 = float(items[14])

                    rsrp = -150
                    cell_id = None
                    freq = None
                    cell_str = ""
                    
                    try:
                        rsrp = float(items[6])
                        cell_id = items[5]
                        freq = int(items[4])
                        cell_str = items[3] + '-' + items[2]
                    except:
                        print(line)
                        continue
                    
                    #if rsrp_50 > -110:
                    
                    if rsrp < -140:
                        continue
                        
                    if operator == 'A' and rsrp > -43.4:
                        continue
                        
                    if operator == 'T' and rsrp > -46.4:
                        continue
                        
                    #if gps_str not in rsrp_grid:
                    #rsrp_grid[gps_str] = [lat, lon, -140, -140, -140]

                    if freq > 2000000:
                        mw_rsrp_list.append(rsrp)
                        #rsrp_grid[gps_str][2] = max(rsrp_50, rsrp_grid[gps_str][2])
                    elif freq > 100000:
                        sub6_rsrp_list.append(rsrp)
                        #rsrp_grid[gps_str][3] = max(rsrp_50, rsrp_grid[gps_str][3])
                    else:
                        lte_rsrp_list.append(rsrp)
                        #rsrp_grid[gps_str][4] = max(rsrp_50, rsrp_grid[gps_str][4])

        '''
        mw_rsrp_list = []
        sub6_rsrp_list = []
        lte_rsrp_list = []
        
        for k, v in rsrp_grid.items():
            mw_rsrp = v[2]
            sub6_rsrp = v[3]
            lte_rsrp = v[4]
            
            if mw_rsrp > -140:
                mw_rsrp_list.append(mw_rsrp)
                
            if sub6_rsrp > -140:
                sub6_rsrp_list.append(sub6_rsrp)
                
            if lte_rsrp > -140:
                lte_rsrp_list.append(lte_rsrp)
                
        '''
                
        mw_rsrp_0 = np.percentile(mw_rsrp_list, 10) if len(mw_rsrp_list) else 0
        mw_rsrp_25 = np.percentile(mw_rsrp_list, 25) if len(mw_rsrp_list) else 0
        mw_rsrp_50 = np.percentile(mw_rsrp_list, 50) if len(mw_rsrp_list) else 0
        mw_rsrp_75 = np.percentile(mw_rsrp_list, 75) if len(mw_rsrp_list) else 0
        mw_rsrp_100 = np.percentile(mw_rsrp_list, 90) if len(mw_rsrp_list) else 0

        sub6_rsrp_0 = np.percentile(sub6_rsrp_list, 10) if len(sub6_rsrp_list) else 0
        sub6_rsrp_25 = np.percentile(sub6_rsrp_list, 25) if len(sub6_rsrp_list) else 0
        sub6_rsrp_50 = np.percentile(sub6_rsrp_list, 50) if len(sub6_rsrp_list) else 0
        sub6_rsrp_75 = np.percentile(sub6_rsrp_list, 75) if len(sub6_rsrp_list) else 0
        sub6_rsrp_100 = np.percentile(sub6_rsrp_list, 90) if len(sub6_rsrp_list) else 0

        lte_rsrp_0 = np.percentile(lte_rsrp_list, 10) if len(lte_rsrp_list) else 0
        lte_rsrp_25 = np.percentile(lte_rsrp_list, 25) if len(lte_rsrp_list) else 0
        lte_rsrp_50 = np.percentile(lte_rsrp_list, 50) if len(lte_rsrp_list) else 0
        lte_rsrp_75 = np.percentile(lte_rsrp_list, 75) if len(lte_rsrp_list) else 0
        lte_rsrp_100 = np.percentile(lte_rsrp_list, 90) if len(lte_rsrp_list) else 0
        
        fout = open(path + "/" + operator + "_rsrp_boxplot_noaggr.csv", 'w')
        fout.write("rat,rsrp_10,rsrp_25,rsrp_50,rsrp_75,rsrp_90\n")
        fout.write('mw' + ',' + str(mw_rsrp_0) + ',' + str(mw_rsrp_25) + ',' + str(mw_rsrp_50) + ',' + str(mw_rsrp_75) + ',' + str(mw_rsrp_100) + '\n')
        fout.write('sub6' + ',' + str(sub6_rsrp_0) + ',' + str(sub6_rsrp_25) + ',' + str(sub6_rsrp_50) + ',' + str(sub6_rsrp_75) + ',' + str(sub6_rsrp_100) + '\n')
        fout.write('lte' + ',' + str(lte_rsrp_0) + ',' + str(lte_rsrp_25) + ',' + str(lte_rsrp_50) + ',' + str(lte_rsrp_75) + ',' + str(lte_rsrp_100) + '\n')
        fout.close()
        
        outlier_mw = []
                
        fout = open(path + "/" + operator + "_mw_rsrp_cdf_noaggr.csv", 'w')
        mw_rsrp_list.sort()
        length = len(mw_rsrp_list)
        cdf_value = 0
        step = 0
        for i in range(length):
            cdf_value += 1/length
            if cdf_value > step:
                step += 0.01
                fout.write(str(mw_rsrp_list[i]) + ','+ str(cdf_value) + '\n')
                if mw_rsrp_list[i] < mw_rsrp_0 or mw_rsrp_list[i] > mw_rsrp_100:
                    outlier_mw.append(mw_rsrp_list[i])
        fout.close()
        
        outlier_sub6 = []
        
        fout = open(path + "/" + operator + "_sub6_rsrp_cdf_noaggr.csv", 'w')
        sub6_rsrp_list.sort()
        length = len(sub6_rsrp_list)
        cdf_value = 0
        step = 0
        for i in range(length):
            cdf_value += 1/length
            if cdf_value > step:
                step += 0.01
                fout.write(str(sub6_rsrp_list[i]) + ','+ str(cdf_value) + '\n')
                if sub6_rsrp_list[i] < sub6_rsrp_0 or sub6_rsrp_list[i] > sub6_rsrp_100:
                    outlier_sub6.append(sub6_rsrp_list[i])
        fout.close()
        
        outlier_lte = []
        
        fout = open(path + "/" + operator + "_lte_rsrp_cdf_noaggr.csv", 'w')
        lte_rsrp_list.sort()
        length = len(lte_rsrp_list)
        cdf_value = 0
        step = 0
        for i in range(length):
            cdf_value += 1/length
            if cdf_value > step:
                step += 0.01
                fout.write(str(lte_rsrp_list[i]) + ','+ str(cdf_value) + '\n')
                if lte_rsrp_list[i] < lte_rsrp_0 or lte_rsrp_list[i] > lte_rsrp_100:
                    outlier_lte.append(lte_rsrp_list[i])
        fout.close()
        
        fout = open(path + "/" + operator + "_outlier_noaggr.csv", 'w')
        for item in outlier_mw:
            fout.write('1,' + str(item) + '\n')
        for item in outlier_sub6:
            fout.write('2,' + str(item) + '\n')
        for item in outlier_lte:
            fout.write('3,' + str(item) + '\n')
        fout.close()


A-R1
56250,39.76537467,-86.15992096,1642896719.69532,174270,199,(MI)Unknown,LTE,nr-ml1

A-R2
A-R3
A-R4
A-R5
V-R6
V-R7
V-R8
V-R9
V-R10
14323,41.88325192,-87.67116149,1634584762.12199,176930,382,(MI)Unknown,LTE,nr-ml1

V-R11
T-R12
T-R13
